
# Imports

In [1]:
import os
import pickle

from music21 import *
from music21 import converter, instrument, note, chord

from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

import numpy as np

Using TensorFlow backend.


# Data Path definition

In [2]:
base_path = "Project/"
training_path = "data/Training/"

preprocessed_file_path = "data/Preprocessed/"

output_base_path = "output/"
baseline_model_result = "baseline_model_result/"

base_model_path = "models/"

os.makedirs(base_path + preprocessed_file_path, exist_ok=True)

os.makedirs(base_path + output_base_path, exist_ok=True)
os.makedirs(base_path + output_base_path + baseline_model_result, exist_ok=True)

os.makedirs(base_path + base_model_path, exist_ok=True)
os.makedirs(base_path + base_model_path + baseline_model_result, exist_ok=True)

# Extracting training files from Google Drive

In [3]:
training_files = [file for file in os.listdir(base_path + training_path) if file.endswith(".mid")]
training_files = [base_path + training_path + file_name for file_name in training_files]
len(training_files)

307

# Preprocessing MIDI Files

In [4]:
notes = []
for idx, file in enumerate(training_files, 1):
    midi = converter.parse(file)

    print("Parsing %s" % file)

    notes_to_parse = None

    try: # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
len(set(notes))

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon - Farewell, Pikachu!.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon - Lavender town.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon - Littleroot Town.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon - Lugias Song.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon - Oracion.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon - Pallet Town.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon - Pkmn Elite 4.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon - pokece

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon Colosseum - Relic Forest.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon Colosseum - Semifinal Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon Colosseum - The Under.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon DiamondPearlPlatinum - Approaching Champion Cynthia.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon DiamondPearlPlatinum - Azelf Mesprit and Uxie Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon DiamondPearlPlatinum - Battle Champion.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon Diamond

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon GoldSilverCrystal - Gym Leader Defeated.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon GoldSilverCrystal - Gym.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon GoldSilverCrystal - Indigo Plateau.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon GoldSilverCrystal - Johto Gym Leader Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon GoldSilverCrystal - Johto Wild Pokemon Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon GoldSilverCrystal - Kanto Gym Leader Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Po

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon Mystery Dungeon BlueRed Rescue Team - Sinister Woods.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon Mystery Dungeon BlueRed Rescue Team - Sky Tower.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon Mystery Dungeon BlueRed Rescue Team - Tiny Woods.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon Mystery Dungeon BlueRed Rescue Team - Title Screen.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon Mystery Dungeon Explorers of TimeDarknessSky - A Sinister Smell Team Skulls Theme.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon Mystery Dungeon Explorers of TimeDarknessSky - A Wish F

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RedBlueYellow - Champion Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RedBlueYellow - Cinnabar Island.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RedBlueYellow - Cycling.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RedBlueYellow - Elite Four.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RedBlueYellow - Ending Theme.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RedBlueYellow - Evolution.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RedBlueYellow - Game Corner Theme.mid
Parsing C:/Users/ch

Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RubySapphireEmerald - Recovery.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RubySapphireEmerald - Rival Battle.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RubySapphireEmerald - Route 101.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RubySapphireEmerald - Route 104.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RubySapphireEmerald - Route 110.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RubySapphireEmerald - Route 111 Desert.mid
Parsing C:/Users/chinm/Documents/Courses/Sem3/NN/Project/projectf_final_zip/projectf/data/Training/Pokemon RubySapphireEmerald - Rout

497

In [5]:
n_notes = len(set(notes))
print(n_notes)

497


# Saving Preprocessed MIDI files

In [6]:
with open(base_path + preprocessed_file_path + "preprocessed_midifiles", "wb") as file_obj:
    pickle.dump(notes, file_obj)

# Read saved preprocessed file

In [7]:
notes = pickle.load(open(base_path + preprocessed_file_path + "preprocessed_midifiles", "rb"))
n_notes= len(set(notes))
print(n_notes)

497


# Get unique pitch names for all notes

In [8]:
pitch_names = sorted(set([item for item in notes]))

len(pitch_names)

497

In [9]:
note_to_int = dict((note, number) for number, note in enumerate(pitch_names))

In [10]:
len(notes)

163685

# Prepare input for NN Models

In [11]:
network_input_list = []
network_output_list = []
sequence_length = 100

In [12]:
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input_list.append([note_to_int[char] for char in sequence_in])
    network_output_list.append(note_to_int[sequence_out])
n_patterns = len(network_input_list)

In [13]:
network_input = np.reshape(network_input_list, (n_patterns, sequence_length, 1))
network_input = network_input/float(n_notes)

network_output = np_utils.to_categorical(network_output_list)

network_input.shape, network_output.shape

((163585, 100, 1), (163585, 497))

# Building Baseline model

In [15]:
from keras.models import Sequential, model_from_json
from keras.layers import Input, Dropout, LSTM, Dense, Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint

In [16]:
def create_network(network_input, n_notes):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_notes))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [17]:
model = create_network(network_input, n_notes)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)      

# Training Baseline Model

In [18]:
history = model.fit(network_input, network_output, epochs=5, batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
100/100 [==============================] - 9s 94ms/step - loss: 6.5301
Epoch 2/5
100/100 [==============================] - 5s 52ms/step - loss: 6.3306
Epoch 3/5
100/100 [==============================] - 6s 56ms/step - loss: 6.7433
Epoch 4/5
100/100 [==============================] - 6s 62ms/step - loss: 6.5717
Epoch 5/5
100/100 [==============================] - 6s 62ms/step - loss: 6.5699


# Generating Musical Notes from Trained Model

In [19]:
def generate_notes(model, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)
    
    # Get pitch names and store in a dictionary
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = np.append(pattern,index)
        #pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [20]:
prediction_output = generate_notes(model, network_input_list, n_notes)

# Creating Midi Files from Output Notes

In [21]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}{}.mid'.format(base_path + output_base_path + baseline_model_result, filename))

In [22]:
create_midi(prediction_output, "lstm_baseline")

In [23]:
# Save Model

model_json = model.to_json()
with open(base_path + base_model_path + baseline_model_result + "lstm_baseline.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(base_path + base_model_path + baseline_model_result + "lstm_baseline.h5")
print("Saved model to disk")


Saved model to disk


In [24]:
# Load Model

# load json and create model
json_file = open(base_path + base_model_path + baseline_model_result + "lstm_baseline.json", 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(base_path + base_model_path + baseline_model_result + "lstm_baseline.h5")
print("Loaded model from disk")

Loaded model from disk
